In [11]:
import json
import os.path as osp
import numpy as np

base_path = '../datasets/CUHK01_original'

with open(osp.join(base_path, 'split.json')) as f:
    split = json.load(f)
#     data = json.load(f)
#     trainval = data['trainval']
#     test_probe = data['test_probe']
#     test_gallery = data['test_gallery']
#     del data

with open(osp.join(base_path, 'meta.json')) as f:
    data = json.load(f)
#     for key, value in data.items():
#         print(key)
    shot = data['shot']
    identities = np.asarray(data['identities'])
    dabase_name = data['name']
    num_cameras = data['num_cameras']
    del data

In [12]:
def _get_list(identities):
    ret = []
    for views in identities:
        for v in views:
            for file in v:
                label = int(osp.basename(file)[:5])
                ret.append((file, label))
    return np.asarray(ret)

val_ratio = 0.2

trainval = identities[split['trainval']]
trainval = _get_list(trainval)
np.random.shuffle(trainval)
num_val = int(len(trainval) * val_ratio)
train = trainval[num_val:]
val = trainval[:num_val]

import os
for i in range(len(train)):
    pathname = train[i][0]
    name = osp.split(pathname)[1][:5]
    filename = pathname.replace('/', '_')
    
    target_path = osp.join(base_path, 'train', name)
    if not osp.exists(target_path):
        os.makedirs(target_path)
        
    os.symlink(osp.abspath(osp.join(base_path, pathname)), osp.join(target_path, filename))

for i in range(len(val)):
    pathname = val[i][0]
    name = osp.split(pathname)[1][:5]
    filename = pathname.replace('/', '_')
    
    target_path = osp.join(base_path, 'val', name)
    if not osp.exists(target_path):
        os.makedirs(target_path)
        
    os.symlink(osp.abspath(osp.join(base_path, pathname)), osp.join(target_path, filename))